In [30]:
# import the necessary packages
import keras
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing import image
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout,Activation
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

In [2]:
from PIL import Image
cur_path = r'E:\Imarticus Learning\PGA14_Deep Learning\Traffic Recognition using CV & DeepLearning'

In [3]:
data = []
labels = []
classes = 43
 
#Retrieving the images and their labels 
for i in range(classes):
    path = os.path.join(cur_path,'train',str(i))
    images = os.listdir(path)

    for a in images:
        try:
            image = Image.open(path + '\\'+ a)
            image = image.resize((30,30))
            image = np.array(image)
    
            #data.append(image)
            #labels.append(i)
            data.append([image,i]) #appending all value together 
        except:
            print("Error loading image")

In [4]:
random.shuffle(data)

In [5]:
print(len(data))

39209


In [6]:
x = []
y = []

for features,label in data:
    x.append(features)
    y.append(label)

In [7]:
#Converting lists into numpy arrays
x = np.array(x)
y = np.array(y)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [9]:
X_train = X_train/255.0
X_val = X_val/255.0

In [10]:
print("Shape of train images is:", X_train.shape)
print("Shape of labels is:", y_train.shape)

Shape of train images is: (31367, 30, 30, 3)
Shape of labels is: (31367,)


In [11]:
#Building the model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(30, 30, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(43))
model.add(Activation('softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
aug = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.15,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest")

model.fit(aug.flow(X_train, y_train, batch_size=32), epochs=20, validation_data=(X_val, y_val))

model.save('my_model.h5')  # always save your weights after training or during training

Epoch 1/20
981/981 [==============================] - 48s 45ms/step - loss: 2.5036 - accuracy: 0.2801 - val_loss: 1.3429 - val_accuracy: 0.5282
Epoch 2/20
981/981 [==============================] - 45s 45ms/step - loss: 1.3458 - accuracy: 0.5487 - val_loss: 0.6452 - val_accuracy: 0.7892
Epoch 3/20
981/981 [==============================] - 40s 41ms/step - loss: 0.8524 - accuracy: 0.7113 - val_loss: 0.3568 - val_accuracy: 0.8846
Epoch 4/20
981/981 [==============================] - 37s 38ms/step - loss: 0.5872 - accuracy: 0.8031 - val_loss: 0.1818 - val_accuracy: 0.9445
Epoch 5/20
981/981 [==============================] - 57s 58ms/step - loss: 0.4274 - accuracy: 0.8566 - val_loss: 0.1052 - val_accuracy: 0.9713
Epoch 6/20
981/981 [==============================] - 45s 46ms/step - loss: 0.3228 - accuracy: 0.8928 - val_loss: 0.0846 - val_accuracy: 0.9762
Epoch 7/20
981/981 [==============================] - 42s 43ms/step - loss: 0.2615 - accuracy: 0.9148 - val_loss: 0.0553 - val_accuracy:

In [13]:
#testing accuracy on test dataset
from sklearn.metrics import accuracy_score

y_test = pd.read_csv('Test.csv')

labels = y_test["ClassId"].values
imgs = y_test["Path"].values

data=[]

for img in imgs:
    image = Image.open(img)
    image = image.resize((30,30))
    data.append(np.array(image))

In [14]:
data[0]

array([[[116, 140, 175],
        [116, 138, 171],
        [119, 138, 173],
        ...,
        [102, 119, 150],
        [101, 122, 149],
        [ 93, 112, 139]],

       [[116, 142, 177],
        [116, 141, 175],
        [117, 141, 174],
        ...,
        [120, 143, 178],
        [122, 144, 176],
        [122, 142, 174]],

       [[118, 142, 174],
        [116, 141, 175],
        [114, 140, 172],
        ...,
        [121, 144, 181],
        [122, 144, 180],
        [119, 142, 178]],

       ...,

       [[117, 137, 168],
        [115, 135, 165],
        [116, 135, 164],
        ...,
        [116, 138, 170],
        [116, 136, 168],
        [118, 139, 171]],

       [[116, 136, 166],
        [115, 134, 167],
        [115, 133, 165],
        ...,
        [114, 135, 168],
        [116, 136, 166],
        [115, 139, 167]],

       [[112, 135, 166],
        [111, 134, 165],
        [119, 135, 166],
        ...,
        [115, 137, 166],
        [117, 138, 167],
        [114, 140, 170]]

In [15]:
X_test=np.array(data)

#pred = model.predict_classes(X_test)  

pred = np.argmax(model.predict(X_test), axis=-1)

#Accuracy with the test data
from sklearn.metrics import accuracy_score
print(accuracy_score(labels, pred))

395/395 [==============================] - 6s 11ms/step
0.9096595407759304


## Here we are testing our model on our saved model also we can test on our 15 epoch data result we got.

In [27]:
from keras.models import load_model
model = load_model('my_model.h5')

In [32]:
path = r'Test\00015.png'

In [33]:
#from keras.preprocessing import image
img = image.load_img(path, target_size = (30,30)) #load the image
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])

value = classes = np.argmax(model.predict(images,batch_size=32),axis=-1) #predict the label for the image

if classes[0]==0:
    print('Speed limit (20km/h)') #print the content

elif classes[0]==1:
        print('Speed limit (30km/h)') #print the content

elif classes[0]==2:
        print('Speed limit (50km/h)') #print the content

elif classes[0]==3:
        print(' Speed limit (60km/h)') #print the content

elif classes[0]==4:
        print('Speed limit (70km/h)') #print the content

elif classes[0]==5:
        print('Speed limit (80km/h)') #print the content

elif classes[0]==6:
        print('End of speed limit (80km/h)') #print the content

elif classes[0]==7:
        print('Speed limit (100km/h)') #print the content

elif classes[0]==8:
        print('Speed limit (120km/h)') #print the content

elif classes[0]==9:
        print('No passing') #print the content
        
elif classes[0]==10:
        print('No passing veh over 3.5 tons') #print the content 
    
elif classes[0]==11:
        print(', Right-of-way at intersection') #print the content
        
elif classes[0]==12:
        print('Priority road') #print the content
        
elif classes[0]==13:
        print('Yield') #print the content       
        
elif classes[0]==14:
        print('Stop') #print the content       
        
elif classes[0]==15:
        print('No vehicles') #print the content
                
elif classes[0]==17:
        print('No entry') #print the content        
                       
elif classes[0]==18:
        print('General caution') #print the content       
                
elif classes[0]==19:
        print('Dangerous curve left') #print the content        
        
elif classes[0]==20:
        print('Dangerous curve right') #print the content        
        
elif classes[0]==21:
        print('Double curve') #print the content       
        
elif classes[0]==23:
        print('Slippery road') #print the content        
        
elif classes[0]==24:
        print('Road narrows on the right') #print the content
        
elif classes[0]==24:
        print('Road narrows on the right') #print the content               
        
elif classes[0]==25:
        print('Road work') #print the content        
        
elif classes[0]==26:
        print('Traffic signals') #print the content
        
elif classes[0]==27:
        print('Pedestrians') #print the content       
                
elif classes[0]==28:
        print('Children crossing') #print the content
        
elif classes[0]==29:
        print( 'Bicycles crossing') #print the content
               
elif classes[0]==30:
        print('Beware of ice/snow') #print the content        
        
elif classes[0]==31:
        print('Wild animals crossing') #print the content        
               
elif classes[0]==32:
        print('End speed + passing limits') #print the content        
        
elif classes[0]==33:
        print('Turn right ahead') #print the content
        
elif classes[0]==34:
        print('Turn left ahead') #print the content        
        
elif classes[0]==35:
        print('Ahead only') #print the content        
               
elif classes[0]==37:
        print('Go straight or left') #print the content        
        
elif classes[0]==38:
        print('Keep right') #print the content
        
elif classes[0]==39:
        print('Keep left') #print the content       
        
elif classes[0]==40:
        print('Roundabout mandatory') #print the content

elif classes[0]==41:
        print('End of no passing') #print the content        
        
else:
    print('End no passing veh > 3.5 tons') #print the content

1/1 [==============================] - 0s 32ms/step
No passing
